In [91]:
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split
import pandas as pd  
import numpy as np  

scaler = MinMaxScaler()
monthly_events = pd.read_csv('./data/monthly_events.csv')
monthly_events['num_events'] = scaler.fit_transform(monthly_events[['num_events']])

# Create sequences for encoder-decoder
def create_sequences(data, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(data) - n_steps_in - n_steps_out):
        X.append(data[i:i + n_steps_in])
        y.append(data[i + n_steps_in:i + n_steps_in + n_steps_out])
    return np.array(X), np.array(y)

# Define the number of input and output steps
n_steps_in, n_steps_out = 12, 3  # use 1 year data as input and predict 3 months

# Create the sequences
X, y = create_sequences(monthly_events['num_events'].values, n_steps_in, n_steps_out)

# Reshape data for LSTM
X = X.reshape((X.shape[0], X.shape[1], 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(241, 12, 1) (61, 12, 1) (241, 3) (61, 3)


In [81]:
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.layers import Input, LSTM, Dense # type: ignore

# Define the input sequence length and number of features
input_seq_length = X.shape[1]
num_features = X.shape[2]
output_seq_length = 3

# Define the encoder input layer
encoder_inputs = Input(shape=(input_seq_length, num_features))

# Define the encoder LSTM layer
encoder_lstm = LSTM(64, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# Discard encoder outputs and only keep the states
encoder_states = [state_h, state_c]

# Define the decoder input layer
decoder_inputs = Input(shape=(output_seq_length, num_features))

# Define the decoder LSTM layer
decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Define the decoder output layer
decoder_dense = Dense(num_features, activation='sigmoid')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model with encoder and decoder inputs and outputs
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='mean_squared_error')

In [82]:
# Fit the model
model.fit([X_train, y_train], y_train, epochs=100, batch_size=32, validation_data=([X_test, y_test], y_test), verbose=0)
# Predict the outputs
y_pred = model.predict([X_test, y_test])


y_pred_df = pd.DataFrame(y_pred.reshape(y_pred.shape[0], y_pred.shape[1]), 
                          columns=[f'PredStep_{i+1}' for i in range(y_pred.shape[1])])
y_pred_df['Sample'] = y_pred_df.index

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step


In [97]:
# Convert X_test to DataFrame
X_test_df = pd.DataFrame(X_test.reshape(X_test.shape[0], X_test.shape[1]), 
                          columns=[f'TimeStep_{i+1}' for i in range(X_test.shape[1])])
X_test_df['Sample'] = X_test_df.index

# Convert y_test to DataFrame
y_test_df = pd.DataFrame(y_test, 
                          columns=[f'FutureStep_{i+1}' for i in range(y_test.shape[1])])
y_test_df['Sample'] = y_test_df.index

test_df = pd.merge(X_test_df, y_test_df, on='Sample')
result_df = pd.merge(test_df, y_pred_df, on='Sample')
result_df.head()

denormalized_result = scaler.inverse_transform(result_df)
denormalized_result_df = pd.DataFrame(denormalized_result, columns=result_df.columns)
denormalized_result_df.drop(columns='Sample', inplace=True)
denormalized_result_df.round(0).head()
denormalized_result_df.shape

(61, 18)

In [101]:
X_df = pd.DataFrame(X.reshape(X.shape[0], X.shape[1]), 
                          columns=[f'TimeStep_{i+1}' for i in range(X.shape[1])])
X_df['Sample'] = X_df.index


y_df = pd.DataFrame(y, columns=[f'FutureStep_{i+1}' for i in range(y.shape[1])])
y_df['Sample'] = y_df.index

# Concatenate the two dataframes on the 'Sample' column
combined_df = pd.merge(X_df, y_df, on='Sample')
combined_df.drop('Sample', axis=1, inplace=True)
combined_results = scaler.inverse_transform(combined_df )
combined_df  = pd.DataFrame(combined_results , columns=combined_df.columns)
combined_df.shape

(302, 15)